In [11]:
import pandas as pd
import nltk
nltk.download('punkt')
from nltk.tokenize import word_tokenize
from sklearn.model_selection import train_test_split
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Flatten
from tensorflow.keras.preprocessing.text import Tokenizer
from keras.layers.convolutional import Conv1D
from keras.layers.convolutional import MaxPooling1D
from tensorflow.keras.layers import Embedding
from tensorflow.keras.preprocessing import sequence
from google.colab import drive
drive.mount('/content/drive')
%cd /content/drive/My Drive/datasets/
!ls

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
/content/drive/My Drive/datasets
beautfulsoup.py        cc.si.300.bin.gz.txt  DataScience	 stopwords.txt
cc.en.300.bin.gz.part  cc.si.300.vec	     file_name.csv	 sufix_list.txt
cc.si.300.bin	       cleaned_data.csv      file.txt
cc.si.300.bin.gz       data.csv		     stem_dictonary.txt


In [2]:

data= pd.read_csv("cleaned_data.csv")

In [3]:

X_train, X_test,y_train, y_test = train_test_split(data['News'],data['Label'],random_state=104,test_size=0.25,shuffle=True)

In [4]:

m=len(word_tokenize(X_train[0]))
for i, row in data.iterrows():
  if(m<len(word_tokenize(row['News']))):
    m=len(word_tokenize(row['News']))

print(m)

19


In [5]:

t = Tokenizer(oov_token='<UNK>')
# fit the tokenizer on the documents
t.fit_on_texts(X_train)
t.word_index['<PAD>'] = 0

In [6]:
X_train = t.texts_to_sequences(X_train)
X_test = t.texts_to_sequences(X_test)

In [7]:
X_train[67]

[1138, 2693, 193, 993, 9, 28, 56, 5477, 9381]

In [8]:
MAX_SEQUENCE_LENGTH = 19

In [12]:
X_train = sequence.pad_sequences(X_train, maxlen=MAX_SEQUENCE_LENGTH)
X_test = sequence.pad_sequences(X_test, maxlen=MAX_SEQUENCE_LENGTH)
X_train.shape, X_test.shape

((24221, 19), (8074, 19))

In [13]:
X_train[0]

array([   0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,   38, 9346, 9347,  180,  128,   32], dtype=int32)

In [14]:

y_train = y_train.to_numpy()
y_test = y_test.to_numpy()

In [19]:
len(t.word_index)

18942

In [20]:
VOCAB_SIZE = len(t.word_index)

In [22]:
EMBED_SIZE = 500
EPOCHS=3
BATCH_SIZE=300
# create the model
model = Sequential()
model.add(Embedding(VOCAB_SIZE, EMBED_SIZE, input_length=MAX_SEQUENCE_LENGTH))
model.add(Conv1D(filters=128, kernel_size=4, padding='same', activation='relu'))
model.add(MaxPooling1D(pool_size=2))
model.add(Conv1D(filters=64, kernel_size=4, padding='same', activation='relu'))
model.add(MaxPooling1D(pool_size=2))
model.add(Conv1D(filters=32, kernel_size=4, padding='same', activation='relu'))
model.add(MaxPooling1D(pool_size=2))
model.add(Flatten())
model.add(Dense(256, activation='softmax'))
model.add(Dense(1, activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_1 (Embedding)     (None, 19, 500)           9471000   
                                                                 
 conv1d_3 (Conv1D)           (None, 19, 128)           256128    
                                                                 
 max_pooling1d_3 (MaxPooling  (None, 9, 128)           0         
 1D)                                                             
                                                                 
 conv1d_4 (Conv1D)           (None, 9, 64)             32832     
                                                                 
 max_pooling1d_4 (MaxPooling  (None, 4, 64)            0         
 1D)                                                             
                                                                 
 conv1d_5 (Conv1D)           (None, 4, 32)            

In [23]:
# Fit the model
model.fit(X_train, y_train, 
          validation_split=0.1,
          epochs=EPOCHS, 
          batch_size=BATCH_SIZE, 
          verbose=1)

Epoch 1/3
73/73 [==============================] - 36s 482ms/step - loss: 0.0000e+00 - accuracy: 0.3059 - val_loss: 0.0000e+00 - val_accuracy: 0.3199
Epoch 2/3
73/73 [==============================] - 36s 499ms/step - loss: 0.0000e+00 - accuracy: 0.3059 - val_loss: 0.0000e+00 - val_accuracy: 0.3199
Epoch 3/3
73/73 [==============================] - 35s 478ms/step - loss: 0.0000e+00 - accuracy: 0.3059 - val_loss: 0.0000e+00 - val_accuracy: 0.3199


In [24]:
scores = model.evaluate(X_test, y_test, verbose=1)
print("Accuracy: %.2f%%" % (scores[1]*100))

253/253 [==============================] - 3s 12ms/step - loss: 0.0000e+00 - accuracy: 0.3091
Accuracy: 30.91%
